In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(googlesheets)
}))


In [2]:
gs_token <- "/home/users/ytanigaw/.googlesheets_token.rds"
gs_auth(token = gs_token)


Auto-refreshing stale OAuth token.


In [3]:
# input

master_phe_info_f <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.20200522.phe.info.tsv'
in_eval_f <- '6_aggregate_eval.20200609-222737.tsv'
GBE_names_sheet <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A'
in_count_f <- '7_count_PTVs_PAVs.20200610-145804.tsv'

# output

annotated_eval_df <- '8_annotate_eval_df.tsv'


In [4]:
GBE_names_df <- GBE_names_sheet %>% gs_url() %>% 
gs_read(ws = 'GBE_names', col_types = cols('Units_of_measurement' = col_character()))


Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Sheet successfully identified: "GBE_names"
Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Accessing worksheet titled 'GBE_names'.


In [5]:
phe_info_df <- fread(master_phe_info_f) %>%
rename('GBE_ID'='#GBE_ID') %>%
filter(str_detect(GBE_ID, '^HC')) %>%
filter(!str_detect(GBE_NAME, '^TTE')) %>%
rename('N_NBW_'='N_GBE', 'N_GBE'='N_NBW') %>%
rename('N_NBW'='N_NBW_')


In [6]:
count_df <- fread(in_count_f) %>%
rename('GBE_ID'='#GBE_ID')

In [7]:
df <- fread(in_eval_f) %>%
rename('GBE_ID'='#GBE_ID') %>%
left_join(
    count_df, by='GBE_ID'
) %>%
left_join(
    GBE_names_df %>%
    select(GBE_ID, GBE_NAME, GBE_short_name),
    by='GBE_ID'
) %>%
left_join(
    phe_info_df %>%
    select(GBE_ID, N, N_GBE, N_NBW, N_AFR, N_EAS, N_SAS),
    by='GBE_ID'
)


In [8]:
df %>% colnames()

[1] "GBE_ID"         "n_variables"    "split"          "geno"          
 [5] "covar"          "geno_covar"     "geno_delta"     "PTVs"          
 [9] "PAVs"           "Others"         "PTVs_novel"     "PAVs_novel"    
[13] "Others_novel"   "GBE_NAME"       "GBE_short_name" "N"             
[17] "N_GBE"          "N_NBW"          "N_AFR"          "N_EAS"         
[21] "N_SAS"

In [10]:
df %>%
select(
    GBE_ID, GBE_short_name, n_variables,
    PTVs, PAVs, Others, PTVs_novel, PAVs_novel, Others_novel,
    split, geno, covar, geno_covar, geno_delta,
    GBE_NAME, N, N_GBE, N_NBW, N_AFR, N_EAS, N_SAS
) %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(annotated_eval_df, sep='\t', na = "NA", quote=F)
